## Here We analyse the data scrapped from the IRDAI website

In [104]:
#All the imports are listed here
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")#Ignoring warnings to clean up the notebook

In [2]:
#Provide the Scrapped data here
data=pd.read_csv('Data/scrapping_data.csv')

/var/folders/2c/p03wl8gx1wv5f3293lgxrn2h0000gn/T/ipykernel_1374/1551312839.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('scrapping_data.csv')


In [6]:
data = data.drop_duplicates().reset_index(drop=True)

In [37]:
data=data[~data['PAN'].isna()].reset_index(drop=True)

In [38]:
#Unique Agents
print(len(set(data['PAN'])))

107153


In [90]:
data.head()

,PAN,Aadhaar,Agent Name,Date of Birth,Insurer Type,Insurer,Agency Code,Date of Appointment,Status of agency,Status Change Date,License No. If any previously held,Insurer_Type,Count,average_vintage,Datetime of Appointment
0,CFBPS4970R,,Gaurav Shanker Sinha,15 Jul 1981,Life,BIRLA SUN LIFE,BH0860,01 Apr 2015,Inforce,01 Apr 2015,,NaN,NaN,NaN,NaN
1,CFBPS4970R,,Mr.GAURAV SHANKER SINHA Ashok Kumar Sinha,,Health,STAR HEALTH AND ALLIED INSURANCE CO.LTD,BA0000154103,01 Apr 2015,Inforce,01 Apr 2015,,NaN,NaN,NaN,NaN
2,ALQPS6808P,,RAM KRIPAL SHAW,19 Mar 1954,Life,LIFE INSURANCE CORPORATION OF INDIA,10029414,01 Apr 2015,Inforce,01 Apr 2015,,NaN,NaN,NaN,NaN
3,ALQPS6808P,,Mr.RAM KRIPAL SHAW LATE.R.N.SHAW,,Health,STAR HEALTH AND ALLIED INSURANCE CO.LTD,BA0000154199,01 Apr 2015,Inforce,01 Apr 2015,,NaN,NaN,NaN,NaN
4,AQJPG0260B,,SUBARNA GHOSH,06 Dec 1976,Life,BHARTI AXA LIFE INSURANCE COMPANY LTD.,BALI134961,01 Apr 2015,Terminated for Non Fulfilment of Minimum busin...,10 Sep 2018,,NaN,NaN,NaN,NaN


In [91]:
#Converting to datetime on a case by case basis because there are multiple formats
data['Datetime of Appointment']=pd.Series([np.nan]*data.shape[0])
for idx,row in data.iterrows():
    try:
        data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%d %b %Y")
    except:
        try:
            data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%d-%m-%Y")
        except:
            try:
                data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%d/%m/%Y")
            except:
                try:
                    data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%d-%b-%Y")
                except:
                    try:
                        data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%d-%b-%y")
                    except:
                        try:
                            data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%d%m%Y")
                        except:
                            try:
                                data.loc[idx,'Datetime of Appointment']=datetime.strptime(row['Date of Appointment'], "%Y-%m-%d")
                            except:
                                data.loc[idx,'Datetime of Appointment']=np.nan

## Calculating average age of agents in the business (Vintage)

In [108]:
#Average Vintage of Agents in the business
data['average_vintage']=pd.Series([np.nan]*data.shape[0])
for idx,row in data.iterrows():
    data.loc[idx,'average_vintage'] = datetime.strptime('21 Mar 2023',"%d %b %Y") - row['Datetime of Appointment']
#Taking Average in a twisted way, because diect isnt possible
(data['average_vintage']-data['average_vintage'].min()).mean() + data['average_vintage'].min()

datetime.timedelta(days=2711)

## Calculating average attrition per company

In [112]:
data['Status of agency'].value_counts()

Inforce                                                           205770
Terminated for Non Fulfilment of Minimum business Guarantee        33067
Resigned                                                           10300
License Expired                                                     5981
Blacklisted                                                          491
Terminated for other disqualification condition (Regulation 7)       374
Cancelled                                                            324
Suspended                                                            270
Death                                                                217
Surrendered                                                           18
Name: Status of agency, dtype: int64

In [116]:
data['Active_Status']=pd.Series([np.nan]*data.shape[0])
for idx,row in data.iterrows():
    if row['Status of agency']=='Inforce':
        data.loc[idx,'Active_Status'] = 'Active'
    else:
        data.loc[idx,'Active_Status'] = 'InActive'

In [137]:
#Calculating average attrition percentage per company
group = data.groupby(['Insurer', 'Active_Status']).agg({'Active_Status':'count'})\
.groupby(level=0).apply(lambda x:100 * x / float(x.sum()))

In [138]:
group

Active_Status
Insurer                                   Active_Status               
AEGON LIFE INSURANCE CO. LTD              Active              7.563025
                                          InActive           92.436975
AVIVA LIFE INSURANCE COMPANY INDIA LTD.   Active             26.998841
                                          InActive           73.001159
Aditya Birla Health Insurance Company Ltd Active             82.089552
...                                                                ...
THE ORIENTAL INSURANCE CO. LTD            InActive            2.468777
UNITED INDIA INSURANCE CO. LTD.           Active             57.733892
                                          InActive           42.266108
UNIVERSAL SOMPO GENERAL INSURANCE CO.LTD  Active             67.272727
                                          InActive           32.727273

[102 rows x 1 columns]

## We see some insurers are about to go out of business since majority of their workforce has left

In [147]:
tmp=group.rename({'Active_Status':'Rate'},axis=1).reset_index()
tmp

,Insurer,Active_Status,Rate
0,AEGON LIFE INSURANCE CO. LTD,Active,7.563025
1,AEGON LIFE INSURANCE CO. LTD,InActive,92.436975
2,AVIVA LIFE INSURANCE COMPANY INDIA LTD.,Active,26.998841
3,AVIVA LIFE INSURANCE COMPANY INDIA LTD.,InActive,73.001159
4,Aditya Birla Health Insurance Company Ltd,Active,82.089552
...,...,...,...
97,THE ORIENTAL INSURANCE CO. LTD,InActive,2.468777
98,UNITED INDIA INSURANCE CO. LTD.,Active,57.733892
99,UNITED INDIA INSURANCE CO. LTD.,InActive,42.266108
100,UNIVERSAL SOMPO GENERAL INSURANCE CO.LTD,Active,67.272727


In [150]:
tmp[tmp['Active_Status']=='InActive'].sort_values(['Rate'], ascending=False).reset_index(drop=True)

,Insurer,Active_Status,Rate
0,FUTURE GENERALI LIFE INSURANCE CO.LTD,InActive,94.572368
1,Apollo Munich Health Insurance Co Ltd,InActive,92.711370
2,AEGON LIFE INSURANCE CO. LTD,InActive,92.436975
3,IDBI Federal Life Insurance Co. Ltd,InActive,91.363636
4,BHARTI AXA LIFE INSURANCE COMPANY LTD.,InActive,90.068966
5,RELIANCE NIPPON LIFE INSURANCE CO.LTD.,InActive,89.815472
6,STAR UNION DAI ICHI LIFE INSURANCE CO. LTD,InActive,87.500000
7,MAGMA HDI GENERAL INSURANCE COMPANY LTD,InActive,82.644628
8,PNB METLIFE INDIA INSURANCE CO. LTD,InActive,81.796502
9,TATA AIA LIFE INSURANCE COMPANY LTD,InActive,79.620581
